# Limpieza de datos del SNIIM (Sistema Nacional de información e Integración de Mercados)

Una vez descargadas todas las tablas del SNIIM en formato CSV, se unifican en un sólo dataframe que contenga toda la información necesaria.

Por ejemplo, un dataframe que unifique a todas las frutas y hortalizas de interés reportadas en el periodo del **01/01/2020 al 31/12/2023** tendrá **más de 1 millón** de observaciones, con periodicidad diaria.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = "./drive/MyDrive/raw_data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importar librerías

In [4]:
import pandas as pd
import os
import re
import numpy as np

## Creación del dataframe con todos los archivos CSV descargados.

Se unifican todos los archivos CSV en un sólo dataframe y se inicia con la limpieza de los datos: creación de nuevas variables, renombramiento de variables, conversión de tipo de datos, etc.

In [5]:
# Se crea un dataframe que contiene la información de todos los archivos CSV
#path = "./raw_data/"
archivos = next(os.walk(path))[2]
#print("Archivos = ", archivos)

lista_df = []

for i in range(len(archivos)):
    df_temp = pd.read_csv(path + archivos[i], thousands=',') # uno de los archivos utiliza commas en sus números

    # Se remueve el row que sólo contiene la categoría del producto "Frutas","Hortalizas"
    df_temp = df_temp.drop([0])

    # Se crea columna nueva
    nombre_producto = re.search(r"(.+)_\d{4}-\d{4}.csv", archivos[i]).group(1).replace("_", " ")
    df_temp['Producto'] = nombre_producto

    lista_df.append(df_temp)

# Se concatenan todos en un sólo dataframe
df_sniim_fyh = pd.concat(lista_df, ignore_index=True)

# Se renombran columnas
df_sniim_fyh.columns= df_sniim_fyh.columns.str.lower()
df_sniim_fyh.rename(columns={"presentación":"presentacion", "precio mín":"precio_min", "precio max":"precio_max", "precio frec":"precio_frec","obs.": "observaciones"}, inplace=True)

# Se convierte variable a tipo datetime
df_sniim_fyh['fecha'] = pd.to_datetime(df_sniim_fyh['fecha'], dayfirst=True, format='%d/%m/%Y')

df_sniim_fyh


,fecha,presentacion,origen,destino,precio_min,precio_max,precio_frec,observaciones,producto
0,2020-01-02,Kilogramo,Sinaloa,"Baja California : Central de Abasto INDIA, Tij...",110.0,110.0,110.0,NaN,Chile California
1,2020-01-03,Kilogramo,Sinaloa,"Baja California : Central de Abasto INDIA, Tij...",110.0,110.0,110.0,NaN,Chile California
2,2020-01-06,Kilogramo,Sinaloa,"Baja California : Central de Abasto INDIA, Tij...",87.0,87.0,87.0,NaN,Chile California
3,2020-01-07,Kilogramo,Sinaloa,"Baja California : Central de Abasto INDIA, Tij...",87.0,87.0,87.0,NaN,Chile California
4,2020-01-08,Kilogramo,Sinaloa,"Baja California : Central de Abasto INDIA, Tij...",87.0,87.0,87.0,NaN,Chile California
...,...,...,...,...,...,...,...,...,...
1016308,2023-10-26,Kilogramo,Sinaloa,Zacatecas: Mercado de Abasto de Zacatecas,12.0,12.0,12.0,NaN,Sandía Rayada
1016309,2023-10-27,Kilogramo,Sinaloa,Zacatecas: Mercado de Abasto de Zacatecas,12.0,12.0,12.0,NaN,Sandía Rayada
1016310,2023-10-30,Kilogramo,Sinaloa,Zacatecas: Mercado de Abasto de Zacatecas,12.0,12.0,12.0,NaN,Sandía Rayada
1016311,2023-10-31,Kilogramo,Sinaloa,Zacatecas: Mercado de Abasto de Zacatecas,12.0,12.0,12.0,NaN,Sandía Rayada


## Análisis básico del dataframe y algunas características.

La mayoría de las variables son cadenas de caracteres a excepción de la fecha y las variables de precios que son flotantes. Ninguna de estas variables presenta valores perdidos.

La variable "**observaciones**" es la única que presenta valores perdidos, más del 95% de sus observaciones son valores perdidos. Se asume que es una variable opcional que sólo se utiliza para indicar algún comentario adicional.

In [6]:
df_sniim_fyh.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016313 entries, 0 to 1016312
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   fecha          1016313 non-null  datetime64[ns]
 1   presentacion   1016313 non-null  object        
 2   origen         1016313 non-null  object        
 3   destino        1016313 non-null  object        
 4   precio_min     1016313 non-null  float64       
 5   precio_max     1016313 non-null  float64       
 6   precio_frec    1016313 non-null  float64       
 7   observaciones  43204 non-null    object        
 8   producto       1016313 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 69.8+ MB


Algunas estadísticas descriptivas sobre las **variables numéricas**:

In [7]:
df_sniim_fyh.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,precio_min,precio_max,precio_frec
count,1016313.00,1016313.00,1016313.00
mean,24.10,26.60,25.30
std,22.20,22.97,22.23
min,0.25,0.35,0.30
25%,12.00,13.89,13.00
50%,17.00,19.81,18.00
75%,29.00,32.00,30.00
max,1222.22,847.50,390.00


Explorando la variable categórica **"producto"** se puede ver el número total de productos (más de 100), y el número de observaciones con las que cuenta cada producto (por ejemplo: para el producto "*Jitomate Saladette*" se encuentan más de 40 mil observaciones.

In [8]:
df_sniim_fyh['producto'].value_counts()

Jitomate Saladette          45824
Manzana Golden Delicious    37937
Papa Alpha                  37258
Manzana Red Delicious       36153
Papaya Maradol              34375
                            ...  
Papa Roja                      23
Melón Cantaloupe # 18         20
Mango Mangloba                 10
Melón Cantaloupe # 17          2
Melón Gotas de miel            1
Name: producto, Length: 122, dtype: int64

No se encuentran **duplicados**:

In [9]:
df_sniim_fyh[df_sniim_fyh.duplicated()]

,fecha,presentacion,origen,destino,precio_min,precio_max,precio_frec,observaciones,producto


Se guarda el dataframe en un archivo CSV.

In [10]:
#path = "./data/"               #activar para guardar local
path = "./drive/MyDrive/data/"  #activar para guardar en google drive
df_sniim_fyh.to_csv(path + 'df_sniim_fyh.csv', index=False)